<a href="https://colab.research.google.com/github/Chantynp/EggsAI/blob/main/Eggs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primero estaremos importando las bibliotecas necesarias y accediendo a la base de datos desde Google Drive

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

datos = pd.read_csv('/content/drive/MyDrive/Carpeta sin título/eggs_data.csv')
datos.rename(columns={datos.columns[0]: 'id'}, inplace=True)
datos.head()


,id,year,week,x,y,eggs
0,1283,2024,35,-89.559975,20.968786,29
1,1284,2024,38,-89.559975,20.968786,29
2,1285,2024,37,-89.559975,20.968786,63
3,1286,2024,36,-89.559975,20.968786,58
4,1287,2024,35,-89.560127,20.968391,17


Ahora se divide el código entre los datos que se usan para la prediccion y el resultado de la predicción.

In [31]:
if '' in datos.columns:
    datos = datos.drop(columns=[''])

X = datos[['year', 'week', 'x', 'y']]
y = datos['eggs']

Creamos distintos archivos de entrenamiento, con un 20% para evaluación y 80% para entrenamiento.

In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)
X_train.head()
#y_train.head()

,year,week,x,y
19231,2024,37,-89.638306,20.916645
9482,2024,37,-89.570673,20.979417
5222,2024,37,-89.666031,21.001734
7810,2024,37,-89.635040,20.987110
16162,2024,35,-89.611997,20.926438


A continuación entrenamos el modelo de machine learning usando XGBRegressor.

In [33]:
modelo = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    reg_alpha=0.5,
    reg_lambda=0.8,
    random_state=42,
    n_jobs=-1
)

modelo.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=0.1, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=2000,
             n_jobs=-1, num_parallel_tree=None, ...)

Evaluamos nuestro modelo predictivo usando las metricas necesarias.

In [37]:
predicciones = modelo.predict(X_valid)
predicciones_int = np.round(predicciones).astype(int)

rmse = np.sqrt(mean_squared_error(y_valid, predicciones_int))
mae = mean_absolute_error(y_valid, predicciones_int)
r2 = r2_score(y_valid, predicciones_int)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.4f}")


RMSE: 37.02
MAE: 19.74
R²: 0.0497


Añadimos el momento donde pueden hacerse predicciones con el modelo aplicando la base de datos que sea.

In [36]:
ruta_usuario = input("Ingresa la ruta del archivo CSV para predicción: ")

datos_usuario = pd.read_csv(ruta_usuario)
datos_usuario.rename(columns={datos_usuario.columns[0]: 'id'}, inplace=True)
X_usuario = datos_usuario[['year', 'week', 'x', 'y']]

pred_usuario = modelo.predict(X_usuario).round().astype(int)

resultados_usuario = pd.DataFrame({
    'id': datos_usuario['id'],
    'eggs_pred': pred_usuario
})

resultados_usuario.to_csv('predicciones_usuario.csv', index=False)
print("Archivo de predicciones guardado como 'predicciones_usuario.csv'")


Ingresa la ruta del archivo CSV para predicción: /content/drive/MyDrive/Carpeta sin título/eggs_data.csv
Archivo de predicciones guardado como 'predicciones_usuario.csv'
